In [232]:
import requests
import re
import json
import pandas as pd
import datetime
import numpy as np

import geopandas

import plotly
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table
from jupyter_dash import JupyterDash

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [178]:
df_full = pd.read_csv('full_dataset.csv')
# df_full.drop_duplicates(subset=['id', 'extraction_date'], inplace=True)
# df_full.to_csv('full_dataset.csv', encoding='utf-8-sig', index=False)

In [151]:
one_day = '2023-06-05'
df_one_day = df_full[df_full['extraction_date'] == one_day]
df_one_day

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,max_level,a,temp_no,temp_date_unit,days_passed,kwd_ypes,enotites
21695,815280481,47153242,Διαμέρισμα 37 τ.μ.,Αθήνα (Κυψέλη),295.0,8,37 τ.μ.,1964.0,['-0.5'],1.0,1.0,πριν από 3 ημέρες,2023-06-05,37.997748,23.739691,NaN,2267911.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Κυψέλη,Διαμέρισμα,37.0,7.972973,[-0.5],-0.5,1,-0.5,-0.5,3,ημ,3,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
21696,815243584,47152200,Διαμέρισμα 70 τ.μ.,Αθήνα (Κολωνάκι),1300.0,19,70 τ.μ.,NaN,"['0.5', '1']",1.0,1.0,πριν από 4 ημέρες,2023-06-05,37.979880,23.739364,NaN,2144692.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Κολωνάκι,Διαμέρισμα,70.0,18.571429,"[0.5, 1.0]",0.5,2,1.5,1.5,4,ημ,4,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
21697,815377532,47158634,Διαμέρισμα 90 τ.μ.,Αθήνα (Γκράβα),460.0,5,90 τ.μ.,NaN,['0.5'],2.0,1.0,πριν από 2 ημέρες,2023-06-05,38.014725,23.741497,NaN,589302.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Γκράβα,Διαμέρισμα,90.0,5.111111,[0.5],0.5,1,0.5,0.5,2,ημ,2,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
21698,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560.0,10,54 τ.μ.,1970.0,"['-1', '0']",1.0,1.0,πριν από 6 ημέρες,2023-06-05,37.979777,23.742884,NaN,2267566.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Λυκαβηττός,Διαμέρισμα,54.0,10.370370,"[-1.0, 0.0]",0.0,2,1.0,1.0,6,ημ,6,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
21699,815330524,47156182,Διαμέρισμα 72 τ.μ.,Αθήνα (Καλλιρρόης),600.0,8,72 τ.μ.,NaN,['1'],2.0,NaN,πριν από 3 ημέρες,2023-06-05,37.964577,23.730672,NaN,562904.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Καλλιρρόης,Διαμέρισμα,72.0,8.333333,[1.0],1.0,1,1.0,1.0,3,ημ,3,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27108,744690391,44614009,Διαμέρισμα 230 τ.μ.,Παπάγου,2100.0,9,230 τ.μ.,1970.0,['1'],3.0,2.0,πριν από 2 μήνες,2023-06-05,37.987296,23.789062,Rokanas Real Estate,32628.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Παπάγου - Χολαργός,NaN,Διαμέρισμα,230.0,9.130435,[1.0],1.0,1,1.0,1.0,2,μή,60,9175,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ
27109,814919233,47136288,Διαμέρισμα 40 τ.μ.,Άγιος Δημήτριος Αττικής (Καλογήρων),410.0,10,40 τ.μ.,NaN,['3'],1.0,1.0,πριν από 1 εβδομάδα,2023-06-05,37.949763,23.738081,NaN,1326944.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Άγιος Δημήτριος,Καλογήρων,Διαμέρισμα,40.0,10.250000,[3.0],3.0,1,3.0,3.0,1,εβ,7,9194,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
27110,806971113,46715091,Διαμέρισμα 60 τ.μ.,Άγιος Δημήτριος Αττικής (Καλογήρων),650.0,11,60 τ.μ.,1980.0,['1'],1.0,1.0,πριν από 2 μήνες,2023-06-05,37.949668,23.737657,Theogenis Real Estate,1503172.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Άγιος Δημήτριος,Καλογήρων,Διαμέρισμα,60.0,10.833333,[1.0],1.0,1,1.0,1.0,2,μή,60,9194,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
27111,804716653,46581752,Διαμέρισμα 78 τ.μ.,Ηλιούπολη,550.0,7,78 τ.μ.,1998.0,['2'],1.0,1.0,πριν από 2 μήνες,2023-06-05,37.950313,23.757343,Cosy House Real Estate & Renovations,1867804.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ηλιούπολη,NaN,Διαμέρισμα,78.0,7.051282,[2.0],2.0,1,2.0,2.0,2,μή,60,9191,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ


In [152]:
df_one_day.groupby('dimos')[['price_per_unit_area']].mean().sort_values('price_per_unit_area', ascending=False)

,price_per_unit_area
dimos,
Γλυφάδα,18.000000
Μαρούσι,14.526316
Φιλοθέης - Ψυχικού,13.532934
Χαλάνδρι,11.000000
Καισαριανή,10.892857
Δάφνη - Υμηττός,10.553571
Άγιος Δημήτριος,10.500000
Αθηναίων,10.355246
Ζωγράφου,9.919271


In [153]:
df_one_day.groupby('dimos').aggregate({'price_per_unit_area': ['mean', 'count', 'min', 'max'], 'construction_year': ['mean', 'min', 'max']}).reset_index().sort_values(by=('price_per_unit_area', 'mean'), ascending=False)

dimos price_per_unit_area                \
                                              mean count min max   
7                      Γλυφάδα           18.000000     1  18  18   
13                     Μαρούσι           14.526316    19   8  20   
19          Φιλοθέης - Ψυχικού           13.532934   167   5  25   
20                    Χαλάνδρι           11.000000    21   8  16   
11                  Καισαριανή           10.892857    84   6  22   
8              Δάφνη - Υμηττός           10.553571    56   5  25   
1              Άγιος Δημήτριος           10.500000     2  10  11   
3                     Αθηναίων           10.355246  3879   4  25   
9                     Ζωγράφου            9.919271   384   4  24   
5                      Βύρωνας            9.889831   118   5  19   
12                    Καλλιθέα            9.466019   309   5  22   
15                  Νέα Σμύρνη            9.242424    66   6  19   
16          Παπάγου - Χολαργός            9.000000     4   8  10   
14                   Νέα Ιωνία            8.647059    17   6  12   
6                      Γαλάτσι            8.500000    94   6  17   
17                   Περιστέρι            8.018182   110   4  15   
4                      Αιγάλεω            8.000000     7   5  13   
2                        Ίλιον            7.475000    40   5  14   
0   Άγιοι Ανάργυροι - Καματερό            7.360000    25   5  12   
18     Φιλαδέλφεια - Χαλκηδόνα            7.357143    14   5  12   
10                   Ηλιούπολη            7.000000     1   7   7   

   construction_year                  
                mean     min     max  
7        2011.000000  2011.0  2011.0  
13       1979.842105  1960.0  1996.0  
19       1983.929078  1948.0  2023.0  
20       1984.823529  1960.0  2005.0  
11       1980.666667  1958.0  2017.0  
8        1987.600000  1954.0  2022.0  
1        1980.000000  1980.0  1980.0  
3        1974.023021  1900.0  2023.0  
9        1978.712871  1958.0  2023.0  
5        1983.652632  1950.0  2023.0  
12       1979.071429  1952.0  2023.0  
15       1977.750000  1945.0  2019.0  
16       1975.000000  1970.0  1985.0  
14       1985.833333  1970.0  2008.0  
6        1984.788732  1970.0  2023.0  
17       1987.243243  1950.0  2023.0  
4        1978.400000  1970.0  1982.0  
2        1991.454545  1950.0  2023.0  
0        1986.800000  1960.0  2017.0  
18       1977.083333  1957.0  2018.0  
10       1998.000000  1998.0  1998.0

In [154]:
df_one_day.groupby('perioxi').aggregate({'price_per_unit_area': ['mean', 'count', 'min', 'max'], 'construction_year': ['mean', 'min', 'max']}).reset_index().sort_values(by=('price_per_unit_area', 'mean'), ascending=False)

perioxi price_per_unit_area                \
                                           mean count min max   
134                Ρηγίλλης           20.000000     8  13  24   
69                 Κολωνάκι           16.248804   209   5  25   
116                   Πλάκα           16.166667    12  12  22   
52                  Ζάππειο           15.750000    20   9  24   
90              Μοναστηράκι           15.666667     9   8  20   
..                      ...                 ...   ...  ..  ..   
37                   Βιαμάξ            6.250000     4   6   7   
6    Άγιος Ιωάννης Θεολόγος            6.000000     9   4   8   
150                   Φλέβα            6.000000     1   6   6   
148                 Τσούμπα            5.500000     2   5   6   
80        Λόφος Αξιωματικών            5.500000     2   5   6   

    construction_year                  
                 mean     min     max  
134       1966.625000  1947.0  2023.0  
69        1967.215909  1930.0  2010.0  
116       1957.363636  1938.0  1970.0  
52        1970.055556  1933.0  2001.0  
90        1957.666667  1901.0  2018.0  
..                ...     ...     ...  
37        1991.500000  1978.0  2005.0  
6         1977.166667  1968.0  1985.0  
150               NaN     NaN     NaN  
148       1965.000000  1960.0  1970.0  
80        1998.000000  1998.0  1998.0  

[156 rows x 8 columns]

In [57]:
# app = JupyterDash(__name__, external_stylesheets=['styles_test.css'])
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app = JupyterDash(__name__)
df = df_full[['type', 'dimos', 'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'extraction_date']]


app.layout = html.Div(
    className="container",
    children=[
        html.H1("Interactive Dashboard"),
        html.Div(
            className="sidebar",
            children=[
                html.Label("Filter 1:"),
                dcc.Dropdown(
                    id='filter1',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df['extraction_date'].unique()
                    ],
                    value=None
                ),
                html.Label("Δήμος!"),
                dcc.Dropdown(
                    id='dimos-filter',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df['dimos'].unique()
                    ],
                    value=None
                ),
            ],
        ),
        html.Div(
            className="content",
            children=[
                dash_table.DataTable(
                    id='table',
                    columns=[{'name': col, 'id': col} for col in df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '0px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                        'textAlign': 'left',
                    },
                ),
            ],
        ),
    ],
)


@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'), 
     Input('dimos-filter', 'value')]
)
def update_table(filter1_value, dimos_filtered):
    if filter1_value is None:
        filtered_df = df
    else:
        filtered_df = df[df['extraction_date'] == filter1_value]
        
    if dimos_filtered:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filtered]
    
    return filtered_df.head(50).to_dict('records')

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [245]:
# app = JupyterDash(__name__, external_stylesheets=['styles_test.css'])
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app = JupyterDash(__name__)
df = df_full[['type', 'dimos', 'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'extraction_date']]


app.layout = html.Div(
    className="container",
    children=[
        html.H1("Interactive Dashboard"),
        html.Div(
            style={
                "display": "inline-block",
                "verticalAlign": "top",
                "width": "20%",
                "padding": "10px",
            },
            children=[
                html.Label("Filter 1:"),
                dcc.Dropdown(
                    id='filter1',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df['extraction_date'].unique()
                    ],
                    value=None
                ),
                html.Label("Δήμος!"),
                dcc.Dropdown(
                    id='dimos-filter',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df['dimos'].unique()
                    ],
                    value=None
                ),
            ],
        ),
        html.Div(
            style={
                "display": "inline-block",
                "width": "70%",
                "padding": "10px",
            },
            children=[
                dash_table.DataTable(
                    id='table',
                    columns=[{'name': col, 'id': col} for col in df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '0px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                        'textAlign': 'left',
                        'userSelect': 'text',
                    },
                ),
                html.Br(),
                dcc.Graph(id='levels'),
            ],
        ),
    ],
)


@app.callback(
    Output('table', 'data'),
    [Input('filter1', 'value'), 
     Input('dimos-filter', 'value')]
)
def update_table(filter1_value, dimos_filtered):
    if filter1_value is None:
        filtered_df = df
    else:
        filtered_df = df[df['extraction_date'] == filter1_value]
        
    if dimos_filtered:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filtered]
    
    return filtered_df.head(50).to_dict('records')

@app.callback(
    Output('levels', 'figure'),
    Input('filter1', 'value')
)
def update_levels(x):
    return fig_levels

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [61]:
df_full[]

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed,MaxDateSeen
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560.0,10,54 τ.μ.,1970.0,"['-1', '0']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Λυκαβηττός,Διαμέρισμα,54.0,10.370370,"[-1.0, 0.0]",0.0,2,2,ημ,2,2023-06-05
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420.0,9,45 τ.μ.,1969.0,['1'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Άνω Πατήσια,Διαμέρισμα,45.0,9.333333,[1.0],1.0,1,2,ημ,2,2023-06-05
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780.0,13,60 τ.μ.,1975.0,['2'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κουντουριώτικα,Διαμέρισμα,60.0,13.000000,[2.0],2.0,1,2,ημ,2,2023-06-05
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600.0,11,55 τ.μ.,1981.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πολύγωνο,Διαμέρισμα,55.0,10.909091,[1.0],1.0,1,3,ημ,3,2023-06-05
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790.0,16,50 τ.μ.,1970.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Πανόρμου,Διαμέρισμα,50.0,15.800000,[1.0],1.0,1,3,ημ,3,2023-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27258,632024644,41302418,Διαμέρισμα 64 τ.μ.,Βύρωνας,640.0,10,64 τ.μ.,NaN,['2'],2.0,NaN,πριν από 2 μήνες,2023-06-05,37.959404,23.750014,NaN,98213.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,NaN,Διαμέρισμα,64.0,10.000000,[2.0],2.0,1,2,μή,60,2023-06-05
27259,798078305,46276831,Διαμέρισμα 150 τ.μ.,Βύρωνας,1050.0,7,150 τ.μ.,1974.0,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-05,37.955561,23.750699,Homenet Real Estate & Development,1161699.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,NaN,Διαμέρισμα,150.0,7.000000,[3.0],3.0,1,2,μή,60,2023-06-05
27260,799256815,46334139,Διαμέρισμα 150 τ.μ.,Ζωγράφου (Κέντρο),1000.0,7,150 τ.μ.,NaN,['3'],3.0,1.0,πριν από 2 μήνες,2023-06-05,37.976858,23.770422,Dimit Estate,1144164.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ζωγράφου,Κέντρο,Διαμέρισμα,150.0,6.666667,[3.0],3.0,1,2,μή,60,2023-06-05
27261,2465980,21365332,Διαμέρισμα 50 τ.μ.,Βύρωνας (Αγορά),590.0,12,50 τ.μ.,1954.0,['1'],1.0,NaN,πριν από 2 μήνες,2023-06-05,37.959260,23.756369,NaN,98222.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Βύρωνας,Αγορά,Διαμέρισμα,50.0,11.800000,[1.0],1.0,1,2,μή,60,2023-06-05


In [155]:
fig = px.bar(df_full[df_full['extraction_date'] == "2023-06-05"].groupby('dimos')['price_per_area'].mean().reset_index(), x='dimos', y='price_per_area')
fig.show()

In [47]:
df_full[df_full['min_level']== -1]

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed,is_new
236,812084310,46975125,Διαμέρισμα 44 τ.μ.,Αθήνα (Κυψέλη),360.0,8,44 τ.μ.,1978.0,['-1'],1.0,1.0,πριν από 2 μήνες,2023-06-01,38.002213,23.738355,14 Real Estate,2206897.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κυψέλη,Διαμέρισμα,44.0,8.181818,[-1.0],-1.0,1,2,μή,60,False
246,813671459,47067408,Διαμέρισμα 47 τ.μ.,Αθήνα (Γκύζη),370.0,8,47 τ.μ.,1975.0,['-1'],1.0,1.0,πριν από 3 εβδομάδες,2023-06-01,37.993728,23.749902,14 Real Estate,2206897.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Γκύζη,Διαμέρισμα,47.0,7.872340,[-1.0],-1.0,1,3,εβ,21,False
284,812861960,47017914,Διαμέρισμα 30 τ.μ.,Αθήνα (Κυψέλη),370.0,12,30 τ.μ.,1975.0,['-1'],1.0,1.0,πριν από 4 εβδομάδες,2023-06-01,38.002045,23.738079,14 Real Estate,2206897.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κυψέλη,Διαμέρισμα,30.0,12.333333,[-1.0],-1.0,1,4,εβ,28,False
665,814163083,47102446,Διαμέρισμα 55 τ.μ.,Αθήνα (Κολωνός),260.0,5,55 τ.μ.,1972.0,['-1'],1.0,1.0,πριν από 2 εβδομάδες,2023-06-01,37.990911,23.712937,Κτηματομεσιτική Στέγη,289992.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κολωνός,Διαμέρισμα,55.0,4.727273,[-1.0],-1.0,1,2,εβ,14,False
802,813713697,47069027,Διαμέρισμα 20 τ.μ.,Αθήνα (Κυψέλη),230.0,12,20 τ.μ.,1957.0,['-1'],1.0,1.0,πριν από 2 εβδομάδες,2023-06-01,38.000954,23.742545,Πάστρας Δημήτριος,88951.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθήνα,Κυψέλη,Διαμέρισμα,20.0,11.500000,[-1.0],-1.0,1,2,εβ,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26021,663436536,42231517,Διαμέρισμα 42 τ.μ.,Ζωγράφου (Άνω Ιλίσια),370.0,9,42 τ.μ.,1970.0,['-1'],1.0,1.0,πριν από 2 μήνες,2023-06-05,37.974304,23.772448,Kappa Property Deals & Management,1929310.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ζωγράφου,Άνω Ιλίσια,Διαμέρισμα,42.0,8.809524,[-1.0],-1.0,1,2,μή,60,False
26041,813947828,47088033,Διαμέρισμα 67 τ.μ.,Καισαριανή (Παγκράτι),390.0,6,67 τ.μ.,1976.0,['-1'],2.0,1.0,πριν από 3 εβδομάδες,2023-06-05,37.967787,23.753820,V&M Properties,1927801.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Καισαριανή,Παγκράτι,Διαμέρισμα,67.0,5.820896,[-1.0],-1.0,1,3,εβ,21,False
26375,815110999,47146047,Διαμέρισμα 93 τ.μ.,Νέα Σμύρνη (Άγιος Σώστης),650.0,7,93 τ.μ.,1970.0,['-1'],2.0,1.0,πριν από 5 ημέρες,2023-06-05,37.949290,23.712566,Politexnoupoli,1242180.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Νέα Σμύρνη,Άγιος Σώστης,Διαμέρισμα,93.0,6.989247,[-1.0],-1.0,1,5,ημ,5,False
26469,790407881,45931359,Διαμέρισμα 34 τ.μ.,Ζωγράφου (Κέντρο),250.0,7,34 τ.μ.,2019.0,['-1'],1.0,1.0,πριν από 2 μήνες,2023-06-05,37.975523,23.770909,Κτήμα-Νετ Κατασκευαστική,2056424.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Ζωγράφου,Κέντρο,Διαμέρισμα,34.0,7.352941,[-1.0],-1.0,1,2,μή,60,False


In [5]:
df_full["is_new"] = df_full["days_passed"] == 0

In [25]:
df_population = df_full.groupby('extraction_date')[['is_new']].aggregate({'is_new':['sum', 'count']}).reset_index()
df_population.columns = ['extraction_date', 'new', 'count']
df_population
df_population['left'] = df_population['new'] + df_population['count'] - df_population['count'].shift(1)
df_population['left'] = df_population['left'].fillna(0).astype('int')
df_population['core'] = df_population['count'] - df_population['new'] - df_population['left']
df_population

,extraction_date,new,count,left,core
0,2023-06-01,226,5456,0,5230
1,2023-06-02,195,5435,174,5066
2,2023-06-03,188,5469,222,5059
3,2023-06-04,40,5457,28,5389
4,2023-06-05,123,5446,112,5211


In [157]:
daily_ids = {date: set(df_full.loc[df_full['extraction_date'] == date, 'id']) for date in df_full['extraction_date'].unique()}
# daily_ids_df = pd.from_dict(daily_ids)
df_population = pd.DataFrame(daily_ids.items(), columns=['extraction_date','ids'])
df_population['ids_yesterday'] = df_population['ids'].shift(1)
df_population.dropna(inplace=True)
df_population['in'] = (df_population['ids'] - df_population['ids_yesterday']).apply(len)
df_population['out'] = (df_population['ids_yesterday'] - df_population['ids']).apply(len)
df_population

,extraction_date,ids,ids_yesterday,in,out
1,2023-06-02,"{814874624, 815104013, 815267868, 812941342, 8...","{814874624, 815104013, 812941342, 815038502, 8...",296,300
2,2023-06-03,"{814874624, 801701890, 815104013, 815267868, 8...","{814874624, 815104013, 815267868, 812941342, 8...",229,200
3,2023-06-04,"{814874624, 801701890, 815104013, 815267868, 8...","{814874624, 801701890, 815104013, 815267868, 8...",107,114
4,2023-06-05,"{814874624, 801701890, 815104013, 33095702, 81...","{814874624, 801701890, 815104013, 815267868, 8...",189,203


In [169]:
fig_bar_population_new_left = px.bar(df_population, x='extraction_date', y=['in', 'out'], orientation='v', 
                                     barmode="group")
fig_bar_population_new_left.update_traces(marker_color='darkcyan', selector=dict(type='bar', name='in'))
fig_bar_population_new_left.update_traces(marker_color='darkturquoise', selector=dict(type='bar', name='out'))

fig_bar_population_new_left.update_traces(text=df_population['in'], textposition='auto', selector=dict(type='bar', name='in'))
fig_bar_population_new_left.update_traces(text=df_population['out'], textposition='auto', selector=dict(type='bar', name='out'))

fig_bar_population_new_left.update_layout(
    title="Incoming and outgoing ads",
    title_font=dict(size=24),
    title_x=0.5
)

fig_bar_population_new_left.update_layout(
    xaxis_title='Date',
    yaxis_title='Ad Count'
)

fig_bar_population_new_left.show()

In [26]:
fig = px.bar(df_population, x='extraction_date', y=['core', 'new', 'left'], orientation='v', title='Ads')
fig.show()

In [6]:
fig = px.bar(df_full.groupby('extraction_date')['is_new'].sum().reset_index(), x='is_new', y='extraction_date', 
             orientation='h', title='New ads')
fig.update_traces(marker_color='darkcyan')
fig.update_layout(
    xaxis_title='New ads',
    yaxis_title='Date'
)

fig.show()

In [7]:
fig = px.line(df_full.groupby('extraction_date')['is_new'].sum().reset_index(), 
              x='extraction_date', y='is_new', title='New ads')
fig.update_traces(line_color='darkcyan')
fig.update_layout(
    xaxis=dict(tickmode='linear', dtick='D1', tickformat='%d/%m/%Y'),
#     yaxis=dict(showgrid=False, zeroline=False),
    xaxis_title='Date',
    yaxis_title='New ads'
)


fig.show()

In [176]:
df_ad_age = df_full[df_full['extraction_date'] == "2023-06-05"].groupby('days_passed').size().reset_index(name='Count')
df_ad_age['age'] = pd.cut(df_ad_age['days_passed'], bins=[0, 7, 14, 21, 28, 60], labels=['< 1 week', '< 2 weeks', '< 3 weeks', '< 1 month', '< 2 months'])
df_age_graph = df_ad_age.groupby('age')['Count'].sum().reset_index(name='Count')
fig_ad_age = px.bar(df_age_graph, x='age', y='Count', 
             orientation='v', title='Ad age (2023-06-05)')
fig_ad_age.update_traces(marker_color='darkcyan')
fig_ad_age.update_traces(text=df_age_graph['Count'], textposition='auto')
fig_ad_age.update_layout(
    xaxis_title='Age',
    yaxis_title='Count',
    title_x=0.5
)

fig_ad_age.show()

In [235]:
df = df_full[(df_full['extraction_date'] == "2023-06-05") & (df_full['level_length'] > 0)].groupby('max_level').aggregate({'price_per_area': ['mean', 'count']}).reset_index()
df.columns = ['max_level', 'price_per_area', 'count']

fig_levels = px.bar(df, x='price_per_area', y='max_level', orientation='h', hover_data='count')

fig_levels.update_traces(marker_color='darkcyan')
fig_levels.update_traces(text=df['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
fig_levels.update_layout(
    xaxis_title='Average price per area',
    yaxis_title='Level',
    title='Prices for different house levels',
    title_x=0.5
)

fig_levels.show()

In [236]:
df = df_full[(df_full['extraction_date'] == "2023-06-05") & (df_full['level_length'] > 0)].groupby('level_length').aggregate({'price_per_area': ['mean', 'count']}).reset_index()
df.columns = ['level_length', 'price_per_area', 'count']
fig_level_length = px.bar(df, x='price_per_area', y='level_length', orientation='h', hover_data='count')

fig_level_length.update_traces(marker_color='darkcyan')
fig_level_length.update_traces(text=df['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
fig_level_length.update_layout(
    xaxis_title='Average price per area',
    yaxis_title='Levels',
    title='Do house levels affect price per area?',
    title_x=0.5
)

fig_level_length.show()

In [226]:
x = df_full[df_full['extraction_date'] == "2023-06-05"].groupby(['bedrooms', 'bathrooms']).aggregate({'price_per_area': ['mean', 'count']}).reset_index()
x.columns = ['bedrooms', 'bathrooms', 'price', 'count']

In [230]:
x[x['count'] > 50] .sort_values('price', ascending=False)

,bedrooms,bathrooms,price,count
3,2.0,2.0,11.816719,201
0,1.0,1.0,10.954192,2024
7,3.0,2.0,10.340443,164
2,2.0,1.0,9.272960,1839
6,3.0,1.0,8.945029,437


In [234]:
# Create JupyterDash app
app = JupyterDash(__name__)

# Define the layout for each page
page1_layout = html.Div([
    html.H1('Page 1'),
    html.P('This is the content of page 1.'),
    html.A('Go to Page 2', href='/page2')
])

page2_layout = html.Div([
    html.H1('Page 2'),
    html.P('This is the content of page 2.'),
])

# Set the app layout using dcc.Location and dcc.Link
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Update the page content based on the URL
@app.callback(
    dash.dependencies.Output('page-content', 'children'),
    [dash.dependencies.Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/page2':
        return page2_layout
    else:
        return page1_layout

# Run the app
app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/

